In [1]:
import numpy as np
from scipy.stats import pearsonr, ttest_1samp
from brainiak.isc import (isc, permutation_isc, bootstrap_isc,
                          phaseshift_isc, timeshift_isc,
                          compute_summary_statistic,
                          _check_timeseries_input, p_from_null)

MAX_RANDOM_SEED = 2**32 - 1

In [2]:
# Simulate correlated data by sampling from multivariate normal
def correlated_data(n_trs, n_subjects, r, mean=0, var=1):
    
    mean = np.full(n_subjects, mean)
    cov = np.full((n_subjects, n_subjects), r)
    np.fill_diagonal(cov, var)
    data = np.random.multivariate_normal(mean, cov, size=n_trs)
    
    return data

In [3]:
# Try a reasonable example with low N and decent correlation
n_trs = 300
n_subjects = 10
r = .3

# Generated our simulated correlated data
data = correlated_data(n_trs, n_subjects, r)

# Get leave-one-out ISC (we expect this to be inflated)
iscs = isc(data, pairwise=False)
mean_isc = compute_summary_statistic(iscs)
print(f"Leave-one-out ISC = {mean_isc}")

# Get pairwise ISC (not inflated)
iscs = isc(data, pairwise=True)
mean_isc = compute_summary_statistic(iscs)
print(f"Pairwise ISC = {mean_isc}")

Leave-one-out ISC = 0.4735845582642008
Pairwise ISC = 0.2857616211356275


In [4]:
def timeflip_isc(data, pairwise=False, summary_statistic='median',
                 n_flips=1000, tolerate_nans=True, random_state=None):

    """Time-series flipping randomization for one-sample ISC
    
    For each voxel or ROI, compute the actual ISC and p-values
    from a null distribution of ISCs where response time series
    are first randomly flipped around zero (i.e. multiplied by 1 or -1).
    Input time-series are mean-centered prior to computing ISC and test.
    If pairwise, apply random time-series flipping to each subject and
    compute pairwise ISCs. If leave-one-out approach is used (pairwise=False),
    apply the random time-series flipping to only the left-out subject in each
    iteration of the leave-one-out procedure. Input data should be a list where
    each item is a time-points by voxels ndarray for a given subject.
    Multiple input ndarrays must be the same shape. If a single ndarray is
    supplied, the last dimension is assumed to correspond to subjects.
    When using leave-one-out approach, NaNs are ignored when computing mean
    time series of N-1 subjects (default: tolerate_nans=True). Alternatively,
    you may supply a float between 0 and 1 indicating a threshold proportion
    of N subjects with non-NaN values required when computing the average time
    series for a given voxel. For example, if tolerate_nans=.8, ISCs will be
    computed for any voxel where >= 80% of subjects have non-NaN values,
    while voxels with < 80% non-NaN values will be assigned NaNs. If set to
    False, NaNs are not tolerated and voxels with one or more NaNs among the
    N-1 subjects will be assigned NaN. Setting tolerate_nans to True or False
    will not affect the pairwise approach; however, if a threshold float is
    provided, voxels that do not reach this threshold will be excluded. Note
    that accommodating NaNs may be notably slower than setting tolerate_nans to
    False. Returns the observed ISC and p-values (two-tailed test), as well as
    the null distribution of ISCs computed on randomly flipped time-series
    data.
    
    Parameters
    ----------
    data : list or ndarray (n_TRs x n_voxels x n_subjects)
        fMRI data for which to compute ISFC

    pairwise : bool, default: False
        Whether to use pairwise (True) or leave-one-out (False) approach

    summary_statistic : str, default: 'median'
        Summary statistic, either 'median' (default) or 'mean'

    n_flips : int, default: 1000
        Number of randomly flipped samples

    tolerate_nans : bool or float, default: True
        Accommodate NaNs (when averaging in leave-one-out approach)

    random_state = int, None, or np.random.RandomState, default: None
        Initial random seed

    Returns
    -------
    observed : float, observed ISC (without time-series flipping)
        Actual ISCs

    p : float, p-value
        p-value based on randomized time-series flipping

    distribution : ndarray, n_flips by voxels
        Time-series flipped null distribution
    """
    
    # Check response time series input format
    data, n_TRs, n_voxels, n_subjects = _check_timeseries_input(data)

    # Mean-center time series for flipping around zero
    data -= np.mean(data, axis=0)

    # Get actual observed ISC
    observed = isc(data, pairwise=pairwise,
                   summary_statistic=summary_statistic,
                   tolerate_nans=tolerate_nans)

    # Roll axis to get subjects in first dimension for loop
    if pairwise:
        data = np.rollaxis(data, 2, 0)

    # Iterate through randomized flips to create null distribution
    distribution = []
    for i in np.arange(n_flips):

        # Random seed to be deterministically re-randomized at each iteration
        if isinstance(random_state, np.random.RandomState):
            prng = random_state
        else:
            prng = np.random.RandomState(random_state)

        # Get a random set of flips based on number of subjects
        flips = prng.choice([-1, 1], size=n_subjects, replace=True)

        # In pairwise approach, apply all flips then compute pairwise ISCs
        if pairwise:

            # Apply flips to each subject's time series
            flipped_data = data * flips

            # Compute null ISC on shifted data for pairwise approach
            flipped_isc = isc(flipped_data, pairwise=pairwise,
                              summary_statistic=summary_statistic,
                              tolerate_nans=tolerate_nans)

        # In leave-one-out, apply flips only to each left-out participant
        elif not pairwise:

            flipped_isc = []
            for s, flip in enumerate(flips):
                flipped_subject = data[..., s] * flip
                nonflipped_mean = np.mean(np.delete(data, s, 2), axis=2)
                loo_isc = isc(np.dstack((flipped_subject, nonflipped_mean)),
                              pairwise=False,
                              summary_statistic=None,
                              tolerate_nans=tolerate_nans)
                flipped_isc.append(loo_isc)

            # Get summary statistics across left-out subjects
            flipped_isc = compute_summary_statistic(
                              np.dstack(flipped_isc),
                              summary_statistic=summary_statistic,
                              axis=2)

        distribution.append(flipped_isc)

        # Update random state for next iteration
        random_state = np.random.RandomState(prng.randint(0, MAX_RANDOM_SEED))

    # Convert distribution to numpy array
    distribution = np.vstack(distribution)

    # Get p-value for actual median from shifted distribution
    p = p_from_null(observed, distribution,
                    side='two-sided', exact=False,
                    axis=0)

    return observed, p, distribution

In [5]:
# Function for running FPR simulations
def fpr_simulation(n_trs, n_subjects, test, r=.0, pairwise=False,
                   nominal_p = .05, n_simulations=1000, seed=None,
                   verbose=True):

    np.random.seed(seed)
    p_distribution = []
    for i in np.arange(n_simulations):
        data = correlated_data(n_trs, n_subjects, r)

        if test == bootstrap_isc:
            iscs = isc(data, pairwise=pairwise)
            _, _, p, _ = test(iscs)

        elif test == permutation_isc:
            iscs = isc(data, pairwise=pairwise)
            _, p, _ = test(iscs)

        elif test in (phaseshift_isc, timeshift_isc, timeflip_isc):
            _, p, _ = test(data, pairwise=pairwise)
            
        elif test == ttest_1samp:
            iscs = isc(data, pairwise=pairwise)
            _, p = ttest_1samp(iscs, popmean=0, axis=0)
        
        elif test == 'flip_timeseries':
            zm_D = data - data.mean(0)  # Zero-mean all subj
            nPerm = 1000
            loo_perms = np.zeros(nPerm+1)
            for perm in range(nPerm+1):
                if perm > 0:
                    zm_D *= (np.random.randint(2, size=n_subjects)*2-1)
                loo_perms[perm] = isc(zm_D, pairwise=pairwise).mean()
            p = [(loo_perms[0] < loo_perms[1:]).mean()]

        if verbose and i > 0 and i % 10 == 0:
            print(f"Finished simulation {i}")

        p_distribution.append(p[0])
        
    p_distribution = np.array(p_distribution)
    
    # Compute number of false positives among simulations
    # i.e. number of p-values less than nominal p of 0.05
    fpr = np.sum(p_distribution <= nominal_p) / n_simulations
    
    return fpr, p_distribution

In [32]:
# Set common parameters across all noise simulations
n_trs = 300
n_subjects = 30
r = .0
pairwise = False
nominal_p = .05
n_simulations = 100

In [33]:
# Run the FPR simulation for the bootstrap test
test = bootstrap_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.100 (for nominal p = 0.05) using "bootstrap_isc"


In [34]:
# Run the FPR simulation for the permutation test
test = permutation_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.060 (for nominal p = 0.05) using "permutation_isc"


In [35]:
# Run the FPR simulation for the phase randomization test
test = phaseshift_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.090 (for nominal p = 0.05) using "phaseshift_isc"


In [37]:
# Run the FPR simulation for the circular time-shift test
test = timeshift_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.090 (for nominal p = 0.05) using "timeshift_isc"


In [39]:
# Run the FPR simulation for sign-flipping the timecourses
test = 'flip_timeseries'
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.050 (for nominal p = 0.05) using "flip_timeseries"


In [51]:
# Run the FPR simulation for the time-series flipping test
test = timeflip_isc
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.060 (for nominal p = 0.05) using "timeflip_isc"


In [66]:
# Run the FPR simulation for the parametric t-test
test = ttest_1samp
fpr, p_distribution = fpr_simulation(n_trs, n_subjects, test,
                                     n_simulations=100, seed=1)
print(f'FPR = {fpr:.3f} (for nominal p = {nominal_p}) using "{test.__name__}"')

Finished simulation 10
Finished simulation 20
Finished simulation 30
Finished simulation 40
Finished simulation 50
Finished simulation 60
Finished simulation 70
Finished simulation 80
Finished simulation 90
FPR = 0.110 (for nominal p = 0.05) using "ttest_1samp"
